### Important Note for question1 !
- Please **do not** change the default variable names in this problem, as we will use them in different parts.
- The default variables are initially set to "None".
- You only need to modify code in the "TODO" part. We added a lot of "assertions" to check your code. **Do not** modify them.

In [1]:
# load packages
import numpy as np
import pandas as pd
import time
from sklearn.naive_bayes import GaussianNB

## P1. Load data and plot
### TODO
- Load train and test data, and split them into inputs(trainX, testX) and labels(trainY, testY)




In [2]:
# Use pandas to load q1_train.csv and q1_test.csv
# Each data point has 200 features(X), followed by 1 label(Y)

#### TODO ####
trainX = pd.read_csv('q1_train.csv',index_col=0)
trainY = trainX.iloc[:,200]
trainX = trainX.drop(columns=['200'])


testX = pd.read_csv('q1_test.csv',index_col=0)
testY = testX.iloc[:,200]
testX = testX.drop(columns=['200'])
##############

assert(len(trainX.shape) == 2)
assert(len(trainY.shape) == 1)
assert(trainX.shape[1] == 200)

## P2. Write your Gaussian NB solver
### TODO
- Finish the myNBSolver() function. 
    - Compute P(y == 0) and P(y == 1), saved in "py0" and "py1"
    - Compute mean/variance of trainX for both y = 0 and y = 1, saved in "mean0", "var0", "mean1" and "var1"
        - Each of them should have shape (N_train, M), where N_train is number of train samples and M is number of features.
    - Compute P(xi | y == 0) and P(xi | y == 1), compare and save **binary** prediction in "train_pred" and "test_pred"
    - Compute train accuracy and test accuracy, saved in "train_acc" and "test_acc".
    - Return train accuracy and test accuracy.

In [3]:
def myNBSolver(trainX, trainY, testX, testY):
    
    N_train = trainX.shape[0] #number training samples
    N_test = testX.shape[0]   #number test samples
    M = trainX.shape[1]       #number of features
    
    #### TODO ####
    # Compute P(y == 0) and P(y == 1)
    zero_array = np.where(trainY.astype(int) == 0)[0]
    ones_array = np.where(trainY.astype(int) == 1)[0]

    py0 = len(zero_array)/N_train
    py1 = len(ones_array)/N_train
    
    ##############
    print("Total probablity is %.2f. Should be equal to 1." %(py0 + py1))

    #### TODO ####
    # Compute mean/var for each label
    
    # split your X into two subsets based on label y (0 or 1), and then compute mean and variance of each subset
    X_y0 = trainX.iloc[zero_array]
    X_y1 = trainX.iloc[ones_array]
    mean0 = np.mean(X_y0)
    mean1 = np.mean(X_y1)
    var0 = np.var(X_y0)
    var1 = np.var(X_y1)
    
    ##############
    assert(mean0.shape[0] == M)
    
    #### TODO ####
    # Compute P(xi|y == 0) and P(xi|y == 1), compare and make prediction
    # This part may spend 5 - 10 minutes or even more if you use for loop, so feel free to 
    # print something (like step number) to check the progress
    
    #Compute P(xi | y == 0) and P(xi | y == 1), compare and save binary prediction in "train_pred" and "test_pred"
    
    # calculate P(xi|y=0) and P(xi|y=1)
    train_pred = np.empty((N_train))
    test_pred = np.empty((N_test))
    
    # For train data
    P_xi_y0 = (1/ np.sqrt(2*np.pi*(var0**2))) * np.exp( (-(trainX - mean0)**2)/ (2*var0**2) )
    P_xi_y1 = (1/ np.sqrt(2*np.pi*(var1**2))) * np.exp( (-(trainX - mean1)**2)/ (2*var1**2) )

    # product of probabilities 
    prod_y0 = np.prod(P_xi_y0, axis=1)
    prod_y1 = np.prod(P_xi_y1, axis=1)

    Prob_y1_x = (prod_y1*py1)/ ( (prod_y1*py1) + (prod_y0*py0) )
    Prob_y0_x = (prod_y0*py0)/ ( (prod_y0*py0) + (prod_y1*py1) )

    for i in range(N_train):
        if Prob_y0_x[i] > Prob_y1_x[i]:
            train_pred[i] = 0
        else:
            train_pred[i] = 1
            
    # For test data
    Pt_xi_y0 = (1/ np.sqrt(2*np.pi*(var0**2))) * np.exp( (-(testX - mean0)**2)/ (2*var0**2) )
    Pt_xi_y1 = (1/ np.sqrt(2*np.pi*(var1**2))) * np.exp( (-(testX - mean1)**2)/ (2*var1**2) )

    tprod_y0 = np.prod(Pt_xi_y0, axis=1)
    tprod_y1 = np.prod(Pt_xi_y1, axis=1)

    Prob_y1_tx = (tprod_y1*py1)/ ( (tprod_y1*py1) + (tprod_y0*py0) )
    Prob_y0_tx= (tprod_y0*py0)/ ( (tprod_y0*py0) + (tprod_y1*py1) )

    for i in range(N_test):
        if Prob_y0_tx[i] > Prob_y1_tx[i]:
            test_pred[i] = 0
        else:
            test_pred[i] = 1


    ##############
    assert(train_pred[0] == 0 or train_pred[0] == 1)
    assert(test_pred[0] == 0 or test_pred[0] == 1)
    
    #### TODO ####
    # Compute train accuracy and test accuracy
    test_counter = 0
    train_counter = 0
    
    for i in range(N_train):
        if train_pred[i].astype(int) == trainY[i].astype(int):
            train_counter += 1
            
    for i in range(N_test):
        if test_pred[i].astype(int) == testY[i].astype(int):
            test_counter += 1    
    train_acc = train_counter/N_train
    test_acc = test_counter/N_test
    
    ##############
    
    return train_acc, test_acc

In [4]:
# driver to test your NB solver
train_acc, test_acc = myNBSolver(trainX, trainY, testX, testY)
print("Train accuracy is %.2f" %(train_acc * 100))
print("Test accuracy is %.2f" %(test_acc * 100))

Total probablity is 1.00. Should be equal to 1.
Train accuracy is 92.07
Test accuracy is 91.90


## P3. Test your result using sklearn
### TODO
- Finish the skNBSolver() function. 
     - fit model, make prediction and return accuracy for train and test sets.

In [5]:
def skNBSolver(trainX, trainY, testX, testY):
    
    #### TODO ####
    # fit model
    # make prediction
    # compute accuracy
    
    model = GaussianNB()
    model.fit(trainX,trainY)
    predicted_train = model.predict(trainX) 
    
    predicted_test = model.predict(testX) 
    
    test_counter = 0
    train_counter = 0
    
    for i in range(len(trainX)):
        if predicted_train[i].astype(int) == trainY[i].astype(int):
            train_counter += 1
            
    for i in range(len(testX)):
        if predicted_test[i].astype(int) == testY[i].astype(int):
            test_counter += 1 
            
    sk_train_acc = train_counter/len(trainX)
    sk_test_acc = test_counter/len(testX)
    
    ##############
    return sk_train_acc, sk_test_acc
    

In [6]:
# driver to test skNBSolver
sk_train_acc, sk_test_acc = skNBSolver(trainX, trainY, testX, testY)
print("Train accuracy is %.2f" %(sk_train_acc * 100))
print("Test accuracy is %.2f" %(sk_test_acc * 100))

Train accuracy is 92.22
Test accuracy is 92.05
